In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

2023-12-25 22:34:59.536611: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-25 22:35:00.995707: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 22:35:05.984658: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 22:35:05.984717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 22:35:06.035493: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
df = pd.read_csv('wine-reviews.csv', usecols=['country','description','points','price','variety','winery'])
df = df.dropna(subset=['description','points'])
df['label'] = (df.points >= 90).astype(int)
df =  df[['description','points','label']]

In [3]:
train,valid,test = np.split(df.sample(frac=1),[int(0.8*len(df)), int(0.9*len(df))])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
    df = dataframe.copy()
    labels = df.pop('label')
    df = df['description']
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [5]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(valid)
test_data = df_to_dataset(test)

In [6]:
#Embedding + Model
embedding = 'nnlm-en-dim50'
hub_layer = hub.KerasLayer(hub.load(embedding), dtype=tf.string, trainable=True)

In [7]:
hub_layer(list(train_data)[0][0])

<tf.Tensor: shape=(1024, 50), dtype=float32, numpy=
array([[ 0.18784617, -0.19107637, -0.07237603, ...,  0.05278848,
         0.01869792,  0.09858268],
       [ 0.242909  , -0.2848941 , -0.23573658, ..., -0.00260593,
         0.04618915, -0.10205477],
       [ 0.4516159 , -0.20035018, -0.06972782, ..., -0.02742683,
         0.19686465, -0.02554434],
       ...,
       [ 0.230189  , -0.31195062,  0.00277889, ..., -0.11691706,
         0.05998907,  0.16602597],
       [ 0.15546523,  0.06577007,  0.14196447, ..., -0.07087004,
        -0.03363374,  0.03173432],
       [ 0.31554493,  0.01783893,  0.27045313, ...,  0.00627854,
        -0.04818711,  0.19001988]], dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_data, epochs=5, validation_data=valid_data)

Epoch 1/5
102/102 [==============================] - 44s 383ms/step - loss: 0.6268 - accuracy: 0.6371 - val_loss: 0.5161 - val_accuracy: 0.7808
Epoch 2/5
102/102 [==============================] - 40s 389ms/step - loss: 0.4787 - accuracy: 0.7776 - val_loss: 0.4041 - val_accuracy: 0.8132
Epoch 3/5
102/102 [==============================] - 40s 389ms/step - loss: 0.4033 - accuracy: 0.8235 - val_loss: 0.3804 - val_accuracy: 0.8242
Epoch 4/5
102/102 [==============================] - 40s 389ms/step - loss: 0.3670 - accuracy: 0.8427 - val_loss: 0.3730 - val_accuracy: 0.8281
Epoch 5/5
102/102 [==============================] - 40s 393ms/step - loss: 0.3452 - accuracy: 0.8550 - val_loss: 0.3716 - val_accuracy: 0.8280


In [11]:
model.evaluate(test_data)

13/13 [==============================] - 1s 29ms/step - loss: 0.3787 - accuracy: 0.8281


[0.37865033745765686, 0.828127384185791]

In [13]:
encoder = tf.keras.layers.TextVectorization(max_tokens=2000)
encoder.adapt(train_data.map(lambda text, label:text))

In [25]:
lstm_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim = len(encoder.get_vocabulary()),
        output_dim = 32,
        mask_zero = True
    ),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0,4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [28]:
lstm_history = model.fit(train_data, epochs=5, validation_data=valid_data)

Epoch 1/5
102/102 [==============================] - 36s 356ms/step - loss: 0.3237 - accuracy: 0.8651 - val_loss: 0.3740 - val_accuracy: 0.8314
Epoch 2/5
102/102 [==============================] - 36s 354ms/step - loss: 0.3090 - accuracy: 0.8722 - val_loss: 0.3839 - val_accuracy: 0.8324
Epoch 3/5
102/102 [==============================] - 36s 355ms/step - loss: 0.2911 - accuracy: 0.8782 - val_loss: 0.3894 - val_accuracy: 0.8310
Epoch 4/5
102/102 [==============================] - 36s 354ms/step - loss: 0.2781 - accuracy: 0.8841 - val_loss: 0.3948 - val_accuracy: 0.8311
Epoch 5/5
102/102 [==============================] - 36s 355ms/step - loss: 0.2640 - accuracy: 0.8903 - val_loss: 0.4158 - val_accuracy: 0.8319


In [29]:
lstm_model.evaluate(test_data)

13/13 [==============================] - 1s 58ms/step - loss: 0.6935 - accuracy: 0.4301


[0.6934940218925476, 0.43006616830825806]